In [1]:
import heapq
import math
import os
import re
from collections import defaultdict, deque

import aocd
import numpy as np
from IPython.display import HTML
from scipy.ndimage import convolve

In [2]:
p = aocd.get_puzzle(year=2025, day=7)

In [3]:
def get_data(test_data: bool = False):
    if test_data:
        data = p.examples[0].input_data
    else:
        data = p.input_data
    return data

In [4]:
def process_data(data):
    grid = [list(l) for l in data.split("\n")]
    grid = np.array(grid)
    return grid

In [5]:
print(get_data(test_data=True))

.......S.......
...............
.......^.......
...............
......^.^......
...............
.....^.^.^.....
...............
....^.^...^....
...............
...^.^...^.^...
...............
..^...^.....^..
...............
.^.^.^.^.^...^.
...............


## Part 1

In [6]:
data = get_data(test_data=False)
grid = process_data(data)

In [7]:
%%time
res = 0
beams = grid[0] == "S"

for row in grid:
    new_beams = np.zeros_like(beams, dtype=bool)

    for col, beam in enumerate(beams):
        if not beam:
            continue

        # Move down
        if row[col] == "^":
            # We have a split that should count to the result
            res += 1
            # Split left and right
            for new_col in (col - 1, col + 1):
                if 0 <= new_col < len(row):
                    new_beams[new_col] = True
        else:
            # Continue straight
            new_beams[col] = True

    beams = new_beams

res

CPU times: user 2.81 ms, sys: 688 μs, total: 3.5 ms
Wall time: 3.09 ms


1555

## Part 2 - Directed Acyclic Graph (DAG) approach

In [8]:
data = get_data(test_data=False)
grid = process_data(data)

In [9]:
class DAG:
    def __init__(self):
        self.graph = defaultdict(list)

    def add_edge(self, parent, child):
        """Add a directed edge parent → child."""
        self.graph[parent].append(child)

        # Ensure child is present in the graph, even if it has no children
        if child not in self.graph:
            self.graph[child] = []

    def children(self, node):
        return self.graph[node]

    def count_paths(self, start):
        """Count number of distinct paths from start to all sink nodes."""
        memo = {}

        def dfs(node):
            if node in memo:
                return memo[node]
                
            kids = self.children(node)
            if not kids:  # sink node
                memo[node] = 1
                return 1

            total = sum(dfs(child) for child in kids)
            memo[node] = total
            return total

        return dfs(start)

    def __repr__(self):
        return "\n".join(f"{k} → {v}" for k, v in self.graph.items())

In [10]:
def build_beam_dag(grid):
    dag = DAG()

    # Find starting position
    start_col = int(np.where(grid[0] == "S")[0][0])
    beam_positions = {start_col}
    level = 0

    start = (0, start_col)

    for row in grid:
        level += 1
        new_beam_positions = set()

        for col in beam_positions:
            parent = (level - 1, col)

            if row[col] == "^":
                # Split
                for new_col in (col - 1, col + 1):
                    if 0 <= new_col < len(row):
                        child = (level, new_col)
                        new_beam_positions.add(new_col)
                        dag.add_edge(parent, child)
            else:
                # Continue straight
                child = (level, col)
                new_beam_positions.add(col)
                dag.add_edge(parent, child)

        beam_positions = new_beam_positions

    return dag, start

In [11]:
%%time
dag, start = build_beam_dag(grid)

print("Number of paths:", dag.count_paths(start))

Number of paths: 12895232295789
CPU times: user 5.52 ms, sys: 641 μs, total: 6.16 ms
Wall time: 6.08 ms
